In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2, p3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 70, bias= True),
            nn.ReLU(),
            nn.Linear(70, 70, bias= True),
            nn.ReLU(),
            nn.BatchNorm1d(70),
            nn.Dropout(p = p1),
            nn.Linear(70, 65, bias = True),
            nn.ReLU(),
            nn.Linear(65, 60, bias= True),
            nn.ReLU(),
            nn.Linear(60, 60, bias= True),
            nn.ReLU(),
            nn.BatchNorm1d(60),
            nn.Dropout(p= p2),
            nn.Linear(60, 55, bias= True),
            nn.ReLU(),
            nn.Linear(55, 50, bias= True),
            nn.ReLU(),
            nn.Linear(50, 45, bias= True),
            nn.ReLU(),
            nn.BatchNorm1d(45),
            nn.Dropout(p= p3),
            nn.Linear(45, 40, bias= True),
            nn.ReLU(),
            nn.Linear(40, 35, bias = True),
            nn.ReLU(),
            nn.Linear(35, 25, bias = True),
            nn.ReLU(),
            nn.Linear(25, 20, bias = True),
            nn.ReLU(),
            nn.Linear(20, 15, bias = True),
            nn.ReLU(),
            nn.Linear(15, 10, bias = True),
            nn.ReLU(),
            nn.Linear(10, 5, bias = True)
        )
        self.classify = nn.Sequential(
            nn.Linear(5, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=100, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 100)

In [5]:
model = Network(0.3, 0.4, 0.5)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=70, bias=True)
    (1): ReLU()
    (2): Linear(in_features=70, out_features=70, bias=True)
    (3): ReLU()
    (4): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=70, out_features=65, bias=True)
    (7): ReLU()
    (8): Linear(in_features=65, out_features=60, bias=True)
    (9): ReLU()
    (10): Linear(in_features=60, out_features=60, bias=True)
    (11): ReLU()
    (12): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): Dropout(p=0.4, inplace=False)
    (14): Linear(in_features=60, out_features=55, bias=True)
    (15): ReLU()
    (16): Linear(in_features=55, out_features=50, bias=True)
    (17): ReLU()
    (18): Linear(in_features=50, out_features=45, bias=True)
    (19): ReLU()
    (20): BatchNorm1d(45, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay= 0.00001)

In [7]:
for epoch in range(500):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 3354.4005964398384
loss - 3312.427163004875
loss - 3309.135870575905
loss - 3304.976012289524
loss - 3302.1072465777397
loss - 3300.681672334671
loss - 3300.098206460476
loss - 3297.8582020401955
loss - 3296.639211654663
loss - 3294.0527215600014
loss - 3291.0620310902596
loss - 3286.3524579405785
loss - 3283.7237365841866
loss - 3281.5054998397827
loss - 3278.0625408887863
loss - 3275.3903953433037
loss - 3273.4190098047256
loss - 3271.7306696772575
loss - 3269.89005279541
loss - 3267.6477927565575
loss - 3266.172672212124
loss - 3263.2795248031616
loss - 3262.482297837734
loss - 3260.7339245676994
loss - 3259.112396776676
loss - 3257.55293571949
loss - 3256.171060502529
loss - 3254.837429225445
loss - 3251.877080142498
loss - 3250.7915112376213
loss - 3250.5899205207825
loss - 3249.324480652809
loss - 3247.7417810559273
loss - 3246.1314554214478
loss - 3244.9616725444794
loss - 3244.6356514692307
loss - 3241.300848186016
loss - 3241.926243185997
loss - 3239.9771254062653
loss 

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6437246963562753


In [9]:
from sklearn.metrics import precision_score
print(precision_score(all_preds, all_labels))

0.45851589447189334


In [10]:
from sklearn.metrics import f1_score
print(f1_score(all_preds, all_labels))

0.4972953844598352


In [11]:
from sklearn.metrics import recall_score
print(recall_score(all_preds, all_labels))

0.5432405566600398
